In [7]:
import sys, os
sys.path.append(os.path.abspath("../src"))
import pandas as pd

from data_loader import load_price_data

# Load and Align Prices (Wide Format)
price_dir = "../data/raw_prices"

price_wide = load_price_data(price_dir)
price_wide.to_parquet('../data/processed/price_wide.parquet')

# Coverage Filter (In-Sample Universe)
min_coverage = 0.85
trading_days = price_wide.shape[0]
coverage = price_wide.notna().sum() / trading_days
valid_tickers = coverage[coverage >= min_coverage].index

print(f"{len(valid_tickers)} / {price_wide.shape[1]} tickers passed coverage filter "
      f"({len(valid_tickers) / price_wide.shape[1]:.2%})")
price_filtered = price_wide[valid_tickers]


price_filtered.to_parquet('../data/processed/price_filtered.parquet')
valid_tickers.to_series().to_csv('../data/metadata/universe_tickers.csv', index=False)

450 / 503 tickers passed coverage filter (89.46%)


In [8]:
import sys, os
sys.path.append(os.path.abspath("../src"))
import pandas as pd

from data_loader import load_price_data

# Load raw wide-format prices
price_dir = "../data/raw_prices"
price_wide_raw = load_price_data(price_dir)

# After initial load
original_tickers = set(price_wide_raw.columns)
print(f"Initial universe: {len(original_tickers)} tickers")

# Drop weekends
price_wide = price_wide_raw[price_wide_raw.index.dayofweek < 5]

# Drop duplicate timestamps
price_wide = price_wide[~price_wide.index.duplicated()]

# Drop tickers with all prices ≤ 1
price_wide = price_wide.loc[:, (price_wide > 1).all()]

# After price > 1 filter
post_price_filter = set(price_wide.columns)
dropped_price = original_tickers - post_price_filter
print(f"Dropped due to price ≤ 1 or nonpositive values: {len(dropped_price)} tickers")




# Save cleaned wide-format price data
price_wide.to_parquet('../data/processed/price_wide.parquet')

# Coverage Filter (In-Sample Universe)
min_coverage = 0.85
trading_days = price_wide.shape[0]
coverage = price_wide.notna().sum() / trading_days
valid_tickers = coverage[coverage >= min_coverage].index

# Reference universe size (e.g., 503 tickers in original S&P 500 file)
original_tickers = price_wide_raw.columns
baseline_count = len(original_tickers)
valid_count = len(valid_tickers)
coverage_pct = valid_count / baseline_count

print(f"{valid_count} / {baseline_count} tickers passed full cleaning and coverage filter "
      f"({coverage_pct:.2%})")

# Final filtered price panel
price_filtered = price_wide[valid_tickers]
price_filtered.to_parquet('../data/processed/price_filtered.parquet')

# Save metadata
valid_tickers.to_series().to_csv('../data/metadata/universe_tickers.csv', index=False)

419 / 419 tickers passed coverage filter (100.00%)


In [9]:
419/503

0.8330019880715706